In [1]:
# Ignore 'dask' warning
import pandas as pd
import gcsfs
from google.cloud import storage
from pandas import DataFrame
from IPython.display import HTML
from google.cloud.storage import Blob
import datalab.storage as gcs_datalab

In [2]:
# Setting up constants. All required
project = 'graydon-moving-indicator'
bucket_name = 'graydon-data'

In [3]:
# Initializing bucket
fs = gcsfs.GCSFileSystem(project='graydon-moving-indicator')
gcs = storage.Client()
bucket = gcs.get_bucket(bucket_name)

In [4]:
selected_columns = ['date_month', 'id_company', 'id_branch',
       'is_discontinued',
       'financial_calamity_outcome', 'date_established', 
       'qty_employees', 'year_qty_employees', 'id_company_creditproxy',
       'score_payment_assessment', 'amt_revenue',
       'year_revenue', 'amt_consolidated_revenue', 'year_consolidated_revenue',
       'amt_consolidated_operating_result',
       'year_consolidated_operating_result', 
       'perc_credit_limit_adjustment', 'color_credit_status', 'rat_pd',
       'score_pd','has_increased_risk',
       'is_sole_proprietor', 'code_SBI_2', 'code_SBI_1',
       'qty_address_mutations_total',
       'qty_address_mutations_month', 
       'has_relocated',
       'has_name_change', 'code_discontinuation', 'code_financial_calamity',
       'qty_issued_credit_reports', 'Associate', 'Authorized official', 'Board member', 'Chairman',
       'Commissioner', 'Director', 'Liquidator', 'Major', 'Managing clerk',
       'Managing partner', 'Member of the partnership', 'Miscellaneous',
       'Owner', 'Secretary', 'Secretary/Treasurer', 'Treasurer', 'Unknown',
       'Vice President', 'amt_operating_result', 'code_legal_form', 'date_financial_calamity_started', 
       'date_financial_calamity_stopped', 'date_start', 'from_date_start',
       'qty_stopped_names', 'qty_started_names', 'year_operating_result'       ]

In [5]:
def aggregate_board_members(df):
    """Agregates the number of board members into one feature """    
    col_list_to_sum = ['associate', 'authorized_official', 'board_member', 'chairman', 'commissioner',
       'director', 'liquidator', 'major', 'managing_clerk', 'managing_partner',
       'member_of_the_partnership', 'miscellaneous', 'owner', 'secretary',
       'secretary/treasurer', 'treasurer', 'unknown', 'vice_president']  
    df['total_changeof_board_members_'] = df[col_list_to_sum].sum(axis=1)
    df = df.drop(columns=col_list_to_sum)
    return df

In [6]:
def read_one_month_csv_from_bucket(year, month, last_day_of_month, dir_prefix = '', selected_columns= ''):
    """ Reads one month of data and returns a pandas Df """
    one_month_df = pd.DataFrame()
    dir_prefix = dir_prefix + '/' + year
    print(dir_prefix)
    blob_list = list(bucket.list_blobs(prefix=dir_prefix))    
    for blob in blob_list:
        if month + '-' + last_day_of_month in blob.name:
            print('Processing file: ', blob.name)
            with fs.open('graydon-data/' + blob.name) as f:
                if selected_columns == '' or None:
                    one_month_df = pd.read_csv(f, sep=';')
                else:
                    one_month_df = pd.read_csv(f, sep=';', usecols= selected_columns)
    one_month_df.columns = (one_month_df.columns.str.strip().str.lower(). 
                    str.replace(' ', '_').str.replace('(', '').str.replace(')', '') )
    return one_month_df

In [7]:
#%%time
def read_all_csv_months_yearly_from_bucket_merged(years_to_read_in_list, dir_prefix = '', selected_columns = ''):
    """ Reads a whole year of data and returns a monthly merged pandas Df """
    all_years_merged_df = pd.DataFrame()
    for year in years_to_read_in_list:
        print('Starting with year: ', year)
        dir_prefix = dir_prefix + '/' + year
        blob_list = list(bucket.list_blobs(prefix=dir_prefix))    
        for blob in blob_list:  
            one_month_df = None
            if 'CSV' in blob.name:
                print('Processing file: ', blob.name)
                with fs.open('graydon-data/' + blob.name) as f:
                    one_month_df = pd.read_csv(f, sep=';', usecols= selected_columns)   
                    one_month_df = one_month_df[(one_month_df['is_sole_proprietor'] == 0) ]
                                               # & (one_month_df['is_discontinued'] == 0) 
                    one_month_df.columns = (one_month_df.columns.str.strip().str.lower(). 
                    str.replace(' ', '_').str.replace('(', '').str.replace(')', '') )
                    one_month_df = aggregate_board_members(one_month_df)
                    one_month_df = clean_data_per_year(one_month_df)
                    all_years_merged_df = all_years_merged_df.append(one_month_df)
            print('The number of rows so far is: ', all_years_merged_df.shape[0])
    return all_years_merged_df

In [8]:
def clean_data_per_year(df):
    """Cleans data and returns formatted df"""
    df['date_month'] = pd.to_datetime(df['date_month'])
    df['financial_calamity_outcome'] = df['financial_calamity_outcome'].fillna(-1) 
    df['qty_employees'] = df['qty_employees'].str.strip() 
    #df.loc[df.qty_employees == 'NA', 'qty_employees'] = 0
    #df['qty_employees'] = df['qty_employees'].fillna(0) 
    df['qty_employees'] = df['qty_employees'].astype(str).astype(int)
    df['year_qty_employees'] = df['year_qty_employees'].str.strip()
    df.loc[df.year_qty_employees == 'NA', 'year_qty_employees'] = None
    df['amt_revenue'] = df['amt_revenue'].str.strip() 
    #df.loc[df.amt_revenue == 'NA', 'amt_revenue'] = 0
    df['amt_revenue'] = df['amt_revenue'].astype(str).str.replace(',','.').astype(float)
    df['year_revenue'] = df['year_revenue'].str.strip() 
    #df.loc[df.year_revenue == 'NA', 'year_revenue'] = 0
    df['amt_consolidated_revenue'] = df['amt_consolidated_revenue'].str.strip() 
    #df.loc[df.amt_consolidated_revenue == 'NA', 'amt_consolidated_revenue'] = 0
    df['amt_consolidated_revenue'] = df['amt_consolidated_revenue'].astype(str).str.replace(',','.').astype(float)
    df['year_consolidated_revenue'] = df['year_consolidated_revenue'].str.strip() 
    #df.loc[df.year_consolidated_revenue == 'NA', 'year_consolidated_revenue'] = 0
    df['amt_consolidated_operating_result'] = df['amt_consolidated_operating_result'].str.strip() 
    #df.loc[df.amt_consolidated_operating_result == 'NA', 'amt_consolidated_operating_result'] = 0
    df['amt_consolidated_operating_result'] = df['amt_consolidated_operating_result'].astype(str).str.replace(',','.').astype(float)
    df['year_consolidated_operating_result'] = df['year_consolidated_operating_result'].str.strip() 
    df.loc[df.year_consolidated_operating_result == 'NA', 'year_consolidated_operating_result'] = 0
    df['score_pd'] = df['score_pd'].str.strip() 
    #df.loc[df.score_pd == 'NA', 'score_pd'] = 0
    df['score_pd'] = df['score_pd'].astype(str).str.replace(',','.').astype(float)
    df['has_increased_risk'] = df['has_increased_risk'].astype(bool)
    #df.loc[df.has_increased_risk == None, 'has_increased_risk'] = False
    #df.loc[df.code_sbi_2.isnull(), 'code_sbi_2'] = 0  
    df.loc[df.date_established < '1700-12-31' , 'date_established'] = None
    df['date_established'] = pd.to_datetime(df['date_established'])
    df['amt_operating_result'] = df['amt_operating_result'].str.strip() 
    #df.loc[df.amt_operating_result == 'NA', 'amt_operating_result'] = 0
    df['amt_operating_result'] = df['amt_operating_result'].astype(str).str.replace(',','.').astype(float)
    df['year_operating_result'] = df['year_consolidated_operating_result'].str.strip() 
    #df.loc[df.year_operating_result == 'NA', 'year_operating_result'] = 0
    return df

In [9]:
def save_df_locally(df, dir_prefix, year, as_json= False):
    """ Saves df as json or csv locally on server """
    if as_json:        
        file_path = dir_prefix + '/' + year + '_merged_cleaned.json'
        df.to_json(file_path)
    else:
        file_path =  dir_prefix + '/' + year + '_merged_cleaned.csv'
        df.to_csv(file_path)

In [10]:
jan_2017_df = read_one_month_csv_from_bucket(year= '2017', month= '01', 
                                             last_day_of_month= '31', dir_prefix ='including_scores/unzipped' , 
                                             selected_columns= selected_columns)

including_scores/unzipped/2017
Processing file:  including_scores/unzipped/2017/modelling_2017-01-01_2017-01-31.CSV


/home/mrodriguezlara/graydon/graydon-moving/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (6,9,10,11,16,17,20,35,36,37,43,44,46,51,54) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [13]:
jan_2017_df = aggregate_board_members(jan_2017_df)

In [11]:
jan_2017_df = clean_data_per_year(jan_2017_df)

In [16]:
jan_2017_df.dtypes

date_month                            datetime64[ns]
id_company                                     int64
id_branch                                      int64
date_established                      datetime64[ns]
is_discontinued                               object
code_discontinuation                         float64
code_financial_calamity                       object
date_financial_calamity_started               object
date_financial_calamity_stopped               object
financial_calamity_outcome                   float64
code_legal_form                              float64
qty_employees                                  int64
year_qty_employees                            object
id_company_creditproxy                        object
score_payment_assessment                     float64
amt_revenue                                  float64
year_revenue                                  object
amt_operating_result                         float64
year_operating_result                         

### Reading one year of data

In [9]:
%%time
df_one_year = read_all_csv_months_yearly_from_bucket_merged(dir_prefix ='including_scores/unzipped', 
                                                              selected_columns= selected_columns
                                                              ,years_to_read_in_list=['2017'])

Starting with year:  2017
The number of rows so far is:  0
Processing file:  including_scores/unzipped/2017/modelling_2017-01-01_2017-01-31.CSV


/home/mrodriguezlara/graydon/graydon-moving/lib/python3.5/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (6,9,10,11,16,17,20,35,36,37,43,44,46,51,54) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


The number of rows so far is:  1869701
Processing file:  including_scores/unzipped/2017/modelling_2017-02-01_2017-02-28.CSV
The number of rows so far is:  3745436
Processing file:  including_scores/unzipped/2017/modelling_2017-03-01_2017-03-31.CSV
The number of rows so far is:  5625527
Processing file:  including_scores/unzipped/2017/modelling_2017-04-01_2017-04-30.CSV
The number of rows so far is:  7512392
Processing file:  including_scores/unzipped/2017/modelling_2017-05-01_2017-05-31.CSV
The number of rows so far is:  9403181
Processing file:  including_scores/unzipped/2017/modelling_2017-06-01_2017-06-30.CSV
The number of rows so far is:  11297952
Processing file:  including_scores/unzipped/2017/modelling_2017-07-01_2017-07-31.CSV
The number of rows so far is:  13197749
Processing file:  including_scores/unzipped/2017/modelling_2017-08-01_2017-08-31.CSV
The number of rows so far is:  15101204
Processing file:  including_scores/unzipped/2017/modelling_2017-09-01_2017-09-30.CSV
The n

In [10]:
# Preview of the data 
HTML(DataFrame(df_one_year).head(40).to_html())

,date_month,id_company,id_branch,date_established,is_discontinued,code_discontinuation,code_financial_calamity,date_financial_calamity_started,date_financial_calamity_stopped,financial_calamity_outcome,code_legal_form,qty_employees,year_qty_employees,id_company_creditproxy,score_payment_assessment,amt_revenue,year_revenue,amt_operating_result,year_operating_result,amt_consolidated_revenue,year_consolidated_revenue,amt_consolidated_operating_result,year_consolidated_operating_result,qty_issued_credit_reports,perc_credit_limit_adjustment,color_credit_status,rat_pd,score_pd,has_increased_risk,is_sole_proprietor,code_sbi_2,code_sbi_1,qty_address_mutations_total,qty_address_mutations_month,date_start,from_date_start,has_relocated,qty_started_names,qty_stopped_names,has_name_change,total_num_board_members
0,2017-01-01,3,10079408,1921-03-17,False,NaN,NaN,NaN,NaN,-1.0,5.0,1,2017,3,21.0,3.552530e-316,2009,9.219265e-318,2015,4.954293e-316,2015,2.346812e-318,2015,4.0,15,G,CCC,-4.907,False,False,64.0,NaN,0,0,NaN,NaN,False,0,0,False,0
1,2017-01-01,5,10079416,1740-01-01,False,NaN,NaN,NaN,NaN,-1.0,5.0,9,2017,1064993,20.0,0.000000e+00,0,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,1.0,5,G,A,-4.892,False,False,46.0,NaN,0,0,NaN,NaN,False,0,0,False,0
2,2017-01-01,6,10079424,1874-11-20,False,NaN,F,2013-01-29,NaN,-1.0,5.0,25,2014,6,24.0,4.446591e-317,2011,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,0.0,-100,R,D,0.000,True,False,41.0,NaN,0,0,NaN,NaN,False,0,0,False,0
3,2017-01-01,9,10079432,1897-05-01,False,NaN,NaN,NaN,NaN,-1.0,5.0,45,2017,9,20.0,0.000000e+00,0,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,0.0,25,G,CCC,-4.998,False,False,47.0,NaN,0,0,NaN,NaN,False,0,0,False,0
4,2017-01-01,12,35,1924-08-01,False,NaN,NaN,NaN,NaN,-1.0,5.0,5,2017,12,28.0,0.000000e+00,0,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,0.0,30,G,BB,-5.096,False,False,64.0,NaN,0,0,NaN,NaN,False,0,0,False,0
5,2017-01-01,14,43,1821-05-01,False,NaN,NaN,NaN,NaN,-1.0,5.0,6,2017,14,20.0,0.000000e+00,0,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,0.0,45,G,BBB,-5.005,False,False,64.0,NaN,0,0,NaN,NaN,False,0,0,False,0
6,2017-01-01,17,51,NaT,False,NaN,NaN,NaN,NaN,-1.0,5.0,1,2017,17,20.0,0.000000e+00,0,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,0.0,20,G,A,-5.014,False,False,64.0,NaN,0,0,NaN,NaN,False,0,0,False,0
7,2017-01-01,23,94,1915-11-15,True,7.0,NaN,NaN,NaN,-1.0,5.0,0,None,23,29.0,0.000000e+00,0,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,0.0,-5,R,D,0.000,True,False,46.0,NaN,0,0,NaN,NaN,False,0,0,False,0
8,2017-01-01,25,10079467,1970-12-07,True,7.0,NaN,NaN,NaN,-1.0,5.0,0,None,25,29.0,0.000000e+00,0,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,0.0,0,R,D,0.000,True,False,61.0,NaN,0,0,NaN,NaN,False,0,0,False,0
9,2017-01-01,47,140,1862-01-01,False,NaN,NaN,NaN,NaN,-1.0,5.0,1,2007,47,20.0,0.000000e+00,0,0.000000e+00,NaN,0.000000e+00,0,0.000000e+00,0,0.0,50,G,BBB,-5.122,False,False,70.0,NaN,0,0,NaN,NaN,False,0,0,False,0


In [11]:
# Displaying number of rows and columns
df_one_year.shape

(22729762, 41)

In [15]:
# Saving df locally
save_df_locally(df= df_one_year, dir_prefix= 'files_to_bucket', year= '2017')